# 📈 Apple Stock Time Series Analysis

This notebook performs a complete Time Series Forecasting workflow on Apple Inc.'s historical stock data using ARIMA modeling.

**Steps Covered:**
- Resampling daily prices to monthly averages
- Trend & seasonality decomposition
- Moving average smoothing
- SARIMA model forecasting using auto_arima
- RMSE evaluation & visualization

In [ ]:
# STEP 0: Install compatible versions
!pip uninstall -y numpy pmdarima
!pip install numpy==1.23.5 pmdarima==2.0.3 --quiet

import os
os.kill(os.getpid(), 9)  # Restart runtime

In [ ]:
# STEP 1: Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from pmdarima import auto_arima

In [ ]:
# STEP 2: Load and Resample Data
df = pd.read_csv('/content/AAPL.csv')  # Ensure the dataset is uploaded to Colab
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

monthly_df = df['Adj Close'].resample('M').mean()

# Plot original time series
monthly_df.plot(figsize=(12, 6), title='Monthly Avg Adjusted Close - Apple')
plt.ylabel('Price ($)')
plt.show()

In [ ]:
# STEP 3: Decomposition
decomposition = seasonal_decompose(monthly_df, model='multiplicative')
fig = decomposition.plot()
fig.set_size_inches(12, 8)
plt.show()

In [ ]:
# STEP 4: Moving Averages
monthly_df = pd.DataFrame(monthly_df)
monthly_df['MA_6'] = monthly_df['Adj Close'].rolling(window=6).mean()
monthly_df['MA_12'] = monthly_df['Adj Close'].rolling(window=12).mean()

plt.figure(figsize=(12, 6))
plt.plot(monthly_df['Adj Close'], label='Actual')
plt.plot(monthly_df['MA_6'], label='6-Month MA', linestyle='--')
plt.plot(monthly_df['MA_12'], label='12-Month MA', color='red')
plt.legend(); plt.title('6 & 12-Month Moving Averages')
plt.show()

In [ ]:
# STEP 5: Forecasting with ARIMA
train = monthly_df['Adj Close'][:-12]
test = monthly_df['Adj Close'][-12:]

auto_model = auto_arima(train, seasonal=True, m=12, trace=True,
                        suppress_warnings=True, stepwise=True)

model = ARIMA(train, order=auto_model.order, seasonal_order=auto_model.seasonal_order)
result = model.fit()

forecast = result.forecast(steps=12)
rmse = np.sqrt(mean_squared_error(test, forecast))
print(f"RMSE: {rmse:.2f} USD")

In [ ]:
# STEP 6: Plot Forecast vs Actual
plt.figure(figsize=(12, 6))
plt.plot(train.index, train, label='Training Data')
plt.plot(test.index, test, label='Actual', color='blue')
plt.plot(test.index, forecast, label='Forecast', color='red', linestyle='--')
plt.fill_between(test.index, forecast*0.9, forecast*1.1, alpha=0.2)
plt.title(f"ARIMA Forecast vs Actual (RMSE={rmse:.2f})")
plt.legend()
plt.show()